In [1]:
import Parameteriser
from Simulator import SolutionSimulator
from Composer import SolutionComposer
from rdkit import Chem  

ModuleNotFoundError: No module named 'carl'

In [ ]:
mol = Chem.MolFromSmiles('CCO')
rdk_pmd = Parameteriser.SolutionParameteriser.via_rdkit(mol = mol)

traj_path = f"trajectories/{Md_Experiment_uuid}"
SolutionSimulator.via_openmm(rdk_pmd, file_name = confid, file_path = traj_path,
                             platform = "CUDA", num_steps = 5000 * 500)
print('Simulation finished, composing mdfp...')
traj = md.load(f"trajectories/{Md_Experiment_uuid}/{confid}.h5")
smiles = Chem.MolToSmiles(mol)
mdfp = SolutionComposer.run(traj, rdk_pmd,smiles=smiles)
mdfp = mdfp.get_mdfp()
mdfp_dict = {'mdfp':str(mdfp)}
print('Mdfp composed, saving to database...')
Mdfp_Experiment_uuid = Md_Experiment_uuid
mdfp_conf_uuid = uuid.uuid4()

print('Connecting to database...')
hostname = 'lebanon'
dbname = 'cs_mdfps'
username = 'cschiebroek'
cn = psycopg2.connect(host=hostname, dbname=dbname, user=username)
cur = cn.cursor()
#retrieve conformer from database by confid
cur.execute("insert into cs_mdfps_schema.mdfp_experiment_data values (%s, %s, %s,%s,%s)",(str(confid), str(Mdfp_Experiment_uuid), json.dumps(mdfp_dict),str(mdfp_conf_uuid),Md_Experiment_uuid))
print(mdfp)
cn.commit()
print('Data saved, closing connection...')
cn.close()
print('Connection closed, exiting...')
